### Classification Trees with numerical features

### Datasets used for the problem:

Iris: has three classes and the task is to accurately predict one of the three sub-types of the Iris flower given four different physical features. These features include the length and width of the sepals and the petals. There are a total of 150 instances with each class having 50 instances.

### Growing Decison Trees 
Decision Trees (DTs) are a non-parametric supervised learning method used for classification and regression. The goal of this question in the assignment is to create a model that predicts the value of a target variable by learning simple decision rules inferred from the data features. 

<i>Note: Write in your code only in the place holders where you are instructed to, replacing None.<i>

In [ ]:
# Do not change the code in this cell
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
# Change it to inline to show the plots
%matplotlib inline

### Here is the first look at your dataset and its feature columns

In [ ]:
# Do not change the code in this cell
iris_data = pd.read_csv("iris.csv")

In [ ]:
# Do not change the code in this cell
iris_data.drop("Id", axis=1, inplace=True)

In [ ]:
# Do not change the code in this cell
iris_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# Do not change the code in this cell
iris_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   SepalLengthCm  150 non-null    float64
 1   SepalWidthCm   150 non-null    float64
 2   PetalLengthCm  150 non-null    float64
 3   PetalWidthCm   150 non-null    float64
 4   Species        150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [ ]:
# Do not change the code in this cell
iris_data.describe()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667
std,0.828066,0.433594,1.764420,0.763161
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


### Task
Shuffle the data and change the categorical features mentioned in the species column to numeric

In [ ]:
# Start code here
# Replace the categorical target values in the Species column to numeric
iris_data = iris_data.replace({'Species': {'Iris-setosa': 1, 'Iris-versicolor': 2, 'Iris-virginica': 3}})
# Shuffle the data
iris_data = iris_data.sample(frac = 1)
# End code here

In [ ]:
# Do not change the code in this cell
iris_data.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
143,6.8,3.2,5.9,2.3,3
35,5.0,3.2,1.2,0.2,1
73,6.1,2.8,4.7,1.2,2
83,6.0,2.7,5.1,1.6,2
135,7.7,3.0,6.1,2.3,3


### Task
Time to code your decision tree.

In the following cell, create a node class for your Decision Tree Classifier having the following attributes:
feature_index, threshold, left, right, info_gain, value, where the condition upon which the decision will be based would be defined by feature_index and threshold, while the attributes left and right will be for accessing the left and the right child of a particular node other than the leaf node in the decision tree. info_gain will denote the information gained by the split denoted by the particular decision node. The value attribute will be holding the value of the majority class of the leaf node without having the other attributes. This will help us to determine the class of new data point.

In [ ]:
class Node:  
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        # Start code here
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value
    # End code here

### Task
In the following cell, you will create a Decision Tree Classifier from scratch class having the following attributes: root, min_samples_split, max_depth. Other instructions have been given in doc strings and comments

In [ ]:
class DecisionTreeClassifier:
    def __init__(self, min_samples_split=2, max_depth=2):
        # Start code here
        # Initialize the root of the decision tree to traverse through the decision tree to None
        self.root = Node()
        # initialize the stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        # End code here 
        
        
    def build_tree(self, dataset, curr_depth = 0):
        """
        This will be a recursive function to build the decision tree.
        dataset: The data that you will be using for your assignment
        curr_depth: Current depth of the tree
        Returns the leaf node
        """
        # Start code here
        # Separate the features and targets into two variables X and Y
        X = dataset.drop(['Species'], axis=1)
        Y = dataset['Species']
        # Extract the number of samples and number of features
        num_samples = len(X)
        num_features = len(dataset.columns) - 1
        
        # split until stopping conditions are met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
            # finding the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"] > 0:
                # recur left
                left_subtree = self.build_tree(best_split['dataset_left'], curr_depth + 1)
                # recur right
                right_subtree = self.build_tree(best_split['dataset_right'], curr_depth + 1)
                # return the decision node in the form of a dictionary
                return Node(best_split["feature_index"], best_split["threshold"],
                           left_subtree, right_subtree, best_split["info_gain"])
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # End code here
        return Node(value=leaf_value)
    
        
    def get_best_split(self, dataset, num_samples, num_features):
        """
        Function to find out the best split
        dataset: input data
        num_samples: Number of samples present in the dataset
        num_features: Number of features in the dataset
        Returns the best split
        """
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # Start code here
        # loop over all the features in the data
        for feature_index in range(num_features):
            feature_values = dataset.iloc[:, feature_index]
            # Hint: You can use np.unique function to retrieve the values of the possible threshold
            possible_thresholds = feature_values.unique()
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if children are not null
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y = dataset['Species']
                    left_y = dataset_left['Species']
                    right_y = dataset_right['Species']
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    # update the best split if needed
                    if curr_info_gain > max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
        # End code here

        return best_split
                    
                    
    def split(self, dataset, feature_index, threshold):
        """
        Function to split the data to the left child and right child in the decision tree
        dataset: input data
        feature_index: feature index used to locate the index of the feature in a particular row in the dataset
        threshold: threshold value based on which the split will be calculated
        Returns the left and right datavalues from the dataset
        """
        # Start code here
        # Hint: Use list comprehension to distinguish which values would be present in left and right 
        # subtree on the basis of threshold
        dataset_left = dataset[dataset.iloc[:, feature_index] < threshold]
        dataset_right = dataset[dataset.iloc[:, feature_index] >= threshold]
        # End code here
        return dataset_left, dataset_right
        
        
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        """
        Function to calculate information gain. This function subtracts the combined information 
        of the child node from the parent node.
        parent: value of parent node
        l_child: value of left child node
        r_child: value of right child node
        mode: based on which information gain will be calculated either entropy/gini index
        Returns the information gain
        """
        # Start code here
        # Calculate the relative sizes of the child node with respect to the parent node
        weight_l = self.entropy(l_child) * len(l_child) / len(parent)
        weight_r = self.entropy(r_child) * len(r_child) / len(parent)
        # Calculate gain on the with respect to the information gain parameter which will either be 
        # gini_index or entropy
        if mode == "gini":
            gain = self.gini_index(parent) - weight_l - weight_r
        else:
            gain = self.entropy(parent) - weight_l - weight_r 
        # End code here
        return gain
    
    def entropy(self, y):
        """
        Function to calculate the entropy
        y: target labels
        Returns entropy
        """
        # Start code here
        # Extract the class labels
        class_labels = np.unique(y)
        # Initialize the entropy
        entropy = 0
        # Calculate the entropy
        for cls in class_labels:
            p_cls = len(y[y==cls]) / len(y)
            entropy += (-p_cls*np.log2(p_cls))
        # End code here
        return entropy
    
    
    def gini_index(self, y):
        """
        Function to calculate gini index
        y: target labels
        Returns gini index
        """
        # Extract the class labels
        class_labels = np.unique(y)
        # Initialize the gini_index
        gini = 0
        # Calculate the gini index
        for cls in class_labels:
            p_cls = len(y[y==cls]) / len(y)
            gini += p_cls ** 2
        return 1 - gini
    
    
    def calculate_leaf_value(self, Y):
        """
        Function to compute thr value of leaf node
        Y: target labels
        Returns leaf node value
        """
        # Start code here
        # Return the most occuring element in Y. Hint: you can use lists 
        Y_value = np.argmax(np.bincount(Y))
        return Y_value
        # End code here
    
    def print_tree(self, tree = None, indent = " "):
        """
        Function to print the tree. Use the pre-order traversal method to print the decision tree.
        # Do not make any changes in this function
        """
        
        if not tree:
            tree = self.root
        
        if tree.value is not None:
            print(tree.value)
            
        else:
            print("X " + str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end = " ")
            self.print_tree(tree.left, indent + indent)
            print("%sright" % (indent), end = " ")
            self.print_tree(tree.right, indent + indent)
            
            
    def fit(self, X, Y):
        """
        Function to train the tree.
        X: Features
        Y: Target
        """
        # Start code here
        # Concatenate X, Y to create the dataset and call the build_tree function recursively
        dataset = np.concatenate([X, Y], axis=1)
        dataset = pd.DataFrame(dataset, columns = iris_data.columns)
        self.root = self.build_tree(dataset, 0)
        # End code here
        
    
    def predict(self, X):
        """
        Prediction function to calculate the all the predictions of the matrix of features 
        provided using make_predictions function
        X: Matrix of features
        Returns predictions using the make_predictions function
        """
        # Start code here
        predictions = []
        for x in X:
            predictions.append(self.make_predictions(x, self.root))
        # End code here
        return predictions
    
    
    def make_predictions(self, x, tree):
        """
        Function to predict a single datapoint
        x: data
        tree: current tree
        Returns predictions
        """
        # Start code here
        # return the value if the node is a leaf node
        if tree.value != None:
            return tree.value
        # Extract feature values of a new datapoint at a given feature index
        feature_val = x[tree.feature_index]
        # Recur through left or right subtree 
        if feature_val < tree.threshold:
            return self.make_predictions(x, tree.left)
        return self.make_predictions(x, tree.right)
      # End code here
 

### Evaluating the model

In [ ]:
# Do not change the code in this cell
X = iris_data.iloc[:, :-1].values
Y = iris_data.iloc[:, -1].values.reshape(-1, 1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 41)
classifier = DecisionTreeClassifier(min_samples_split=3, max_depth=3)
classifier.fit(X_train, Y_train)
classifier.print_tree()
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score
print("Accuracy is: "+str(accuracy_score(Y_test, y_pred)))

X 2 <= 3.0 ? 0.909736122531166
 left: 1
 right X 3 <= 1.8 ? 0.7203020886903042
  left: X 2 <= 5.0 ? 0.23172020079568187
    left: X 3 <= 1.7 ? 0.17203694935311378
        left: 2
        right 3
    right X 3 <= 1.6 ? 0.4591479170272448
        left: 3
        right 2
  right 3
Accuracy is: 0.9666666666666667
